In [80]:
# Feature Engineering - Modelo de Fuga Colsubsidio
# ====================================================
# 
# Objetivo: Crear variables derivadas con lógica de negocio crediticio
# - Variables de utilización y comportamiento financiero
# - Índices de stress y actividad del cliente
# - Variables específicas de beneficios Colsubsidio
# - Validación de calidad de features creadas

# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================

import pandas as pd
import numpy as np

# %%
# Cargar datos preparados
train = pd.read_csv('data/processed/train_prepared.csv')
test = pd.read_csv('data/processed/test_prepared.csv')

print(f"Train: {train.shape}")
print(f"Test: {test.shape}")

Train: (50001, 33)
Test: (5001, 32)


In [81]:
# =============================================================================
# CREACIÓN DE FEATURES DE UTILIZACIÓN
# =============================================================================
def crear_features_utilizacion(df):
   df_new = df.copy()
   
   # Utilización del cupo
   df_new['utilizacion_cupo'] = df_new['Saldo'] / (df_new['Limite.Cupo'] + 1)
   
   # Disponibilidad restante
   df_new['disponibilidad_restante'] = df_new['Limite.Cupo'] - df_new['Saldo']
   
   # Ratio avances
   df_new['ratio_avances'] = df_new['Disponible.Avances'] / (df_new['Limite.Cupo'] + 1)
   
   return df_new

train = crear_features_utilizacion(train)
test = crear_features_utilizacion(test)

print("Features utilización creadas")

# %%

Features utilización creadas


In [82]:
# =============================================================================
# CREACIÓN DE FEATURES DE PAGOS
# =============================================================================
def crear_features_pagos(df):
   df_new = df.copy()
   
   # Capacidad de pago
   df_new['capacidad_pago'] = df_new['Pago.del.Mes'] / (df_new['Saldo'] + 1)
   
   # Cumplimiento mínimo
   df_new['cumplimiento_minimo'] = df_new['Pago.del.Mes'] / (df_new['Pago.Minimo'] + 1)
   
   # Tendencia pagos
   df_new['tendencia_pagos'] = (df_new['Pago.del.Mes'] - df_new['Pagos.Mes.Ant']) / (df_new['Pagos.Mes.Ant'] + 1)
   
   return df_new

train = crear_features_pagos(train)
test = crear_features_pagos(test)

print("Features pagos creadas")

# %%

Features pagos creadas


In [83]:
# =============================================================================
#  CREACIÓN DE FEATURES DE ACTIVIDAD
# =============================================================================
def crear_features_actividad(df):
   df_new = df.copy()
   
   # Cliente activo
   df_new['cliente_activo'] = (df_new['Vtas.Mes.Ant'] > 0).astype(int)
   
   # Variación saldo
   df_new['variacion_saldo'] = (df_new['Saldo'] - df_new['Saldos.Mes.Ant']) / (df_new['Saldos.Mes.Ant'] + 1)
   
   # Frecuencia uso (proxy)
   df_new['frecuencia_uso'] = ((df_new['Vtas.Mes.Ant'] > 0) & (df_new['Pago.del.Mes'] > 0)).astype(int)
   
   return df_new

train = crear_features_actividad(train)
test = crear_features_actividad(test)

print("Features actividad creadas")


Features actividad creadas


In [84]:
# =============================================================================
# CREACIÓN DE FEATURES DE ALERTAS
# =============================================================================
def crear_features_alertas(df):
   df_new = df.copy()
   
   # Mora inicial (1-30 días)
   df_new['mora_inicial'] = ((df_new['Edad.Mora'] > 0) & (df_new['Edad.Mora'] <= 30)).astype(int)
   
   # Cliente con mora
   df_new['tiene_mora'] = (df_new['Edad.Mora'] > 0).astype(int)
   
   # Patrón reducción (menos ventas)
   df_new['patron_reduccion'] = (df_new['Vtas.Mes.Ant'] < df_new['Saldos.Mes.Ant'] * 0.1).astype(int)
   
   return df_new

train = crear_features_alertas(train)
test = crear_features_alertas(test)

print("Features alertas creadas")

Features alertas creadas


In [85]:
# =============================================================================
# CREACIÓN DE FEATURES DE PERFIL SOCIODEMOGRÁFICO
# =============================================================================
def crear_features_perfil(df):
   df_new = df.copy()
   
   # Segmento edad riesgo
   df_new['edad_riesgo'] = pd.cut(df_new['edad'], bins=[0, 25, 35, 50, 100], labels=[1, 2, 3, 4]).astype(float)
   
   # Cobertura beneficios
   df_new['cobertura_beneficios'] = df_new['cuota_monetaria'] + df_new['sub_vivenda'] + df_new['bono_lonchera']
   
   # Estrato riesgo
   df_new['estrato_alto'] = (df_new['estrato'] >= 4).astype(int)
   
   return df_new

train = crear_features_perfil(train)
test = crear_features_perfil(test)

print("Features perfil creadas")

# %%

Features perfil creadas


In [86]:
# =============================================================================
# VERIFICAR FEATURES CREADAS
# =============================================================================
nuevas_features = [
   'utilizacion_cupo', 'disponibilidad_restante', 'ratio_avances',
   'capacidad_pago', 'cumplimiento_minimo', 'tendencia_pagos',
   'cliente_activo', 'variacion_saldo', 'frecuencia_uso',
   'mora_inicial', 'tiene_mora', 'patron_reduccion',
   'edad_riesgo', 'cobertura_beneficios', 'estrato_alto'
]

print(f"\nFeatures creadas: {len(nuevas_features)}")
for feature in nuevas_features:
   if feature in train.columns:
       print(f"✓ {feature}")




Features creadas: 15
✓ utilizacion_cupo
✓ disponibilidad_restante
✓ ratio_avances
✓ capacidad_pago
✓ cumplimiento_minimo
✓ tendencia_pagos
✓ cliente_activo
✓ variacion_saldo
✓ frecuencia_uso
✓ mora_inicial
✓ tiene_mora
✓ patron_reduccion
✓ edad_riesgo
✓ cobertura_beneficios
✓ estrato_alto


In [87]:
# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================
def crear_features_beneficios_avanzadas(df):
   df_new = df.copy()
   
   # Engagement con beneficios
   df_new['tiene_beneficios'] = (df_new['cobertura_beneficios'] > 0).astype(int)
   df_new['beneficios_completos'] = (df_new['cobertura_beneficios'] == 3).astype(int)
   
   # Penetración por tipo
   df_new['solo_cuota_monetaria'] = ((df_new['cuota_monetaria'] == 1) & 
                                    (df_new['sub_vivenda'] == 0) & 
                                    (df_new['bono_lonchera'] == 0)).astype(int)
   
   # Perfil beneficiario
   df_new['beneficiario_premium'] = ((df_new['cuota_monetaria'] == 1) & 
                                    (df_new['sub_vivenda'] == 1)).astype(int)
   
   return df_new

train = crear_features_beneficios_avanzadas(train)
test = crear_features_beneficios_avanzadas(test)

print("Features beneficios Colsubsidio completadas")

# %%


Features beneficios Colsubsidio completadas


In [88]:
# =============================================================================
# VALIDACIÓN DE CALIDAD DE FEATURES
# =============================================================================
def validar_calidad_features(df, features_list):
   print("=== VALIDACIÓN CALIDAD FEATURES ===")
   
   # Verificar valores problemáticos
   problemas = []
   for feature in features_list:
       if feature in df.columns:
           infinitos = np.isinf(df[feature]).sum()
           nulos = df[feature].isnull().sum()
           
           if infinitos > 0 or nulos > 0:
               problemas.append(f"{feature}: inf={infinitos}, null={nulos}")
   
   if problemas:
       print("⚠️ Problemas encontrados:")
       for p in problemas:
           print(f"  {p}")
   else:
       print("✅ Sin valores problemáticos")
   
   # Verificar correlaciones altas
   numeric_features = [f for f in features_list if f in df.columns and df[f].dtype in ['int64', 'float64']]
   corr_matrix = df[numeric_features].corr()
   
   print(f"\nCorrelaciones altas (>0.8):")
   alta_corr = []
   for i in range(len(corr_matrix.columns)):
       for j in range(i+1, len(corr_matrix.columns)):
           corr_val = abs(corr_matrix.iloc[i, j])
           if corr_val > 0.8:
               alta_corr.append(f"{corr_matrix.columns[i]} - {corr_matrix.columns[j]}: {corr_val:.3f}")
   
   if alta_corr:
       for corr in alta_corr:
           print(f"  ⚠️ {corr}")
   else:
       print("  ✅ Sin correlaciones problemáticas")

# Todas las features creadas
todas_features = nuevas_features + ['tiene_beneficios', 'beneficios_completos', 'solo_cuota_monetaria', 'beneficiario_premium']
validar_calidad_features(train, todas_features)


=== VALIDACIÓN CALIDAD FEATURES ===
✅ Sin valores problemáticos

Correlaciones altas (>0.8):
  ⚠️ utilizacion_cupo - ratio_avances: 0.813


In [89]:
# =============================================================================
# ESTADÍSTICAS FEATURES CLAVE
# =============================================================================
print("\n=== ESTADÍSTICAS FEATURES CLAVE ===")

features_clave = ['utilizacion_cupo', 'capacidad_pago', 'cliente_activo', 'cobertura_beneficios']

for feature in features_clave:
   if feature in train.columns:
       print(f"\n{feature}:")
       print(f"  Min: {train[feature].min():.3f}")
       print(f"  Max: {train[feature].max():.3f}")
       print(f"  Media: {train[feature].mean():.3f}")
       print(f"  Ceros: {(train[feature] == 0).sum()}")

# %%



=== ESTADÍSTICAS FEATURES CLAVE ===

utilizacion_cupo:
  Min: 0.000
  Max: 4.151
  Media: 0.359
  Ceros: 24047

capacidad_pago:
  Min: 0.000
  Max: 281700.000
  Media: 380.830
  Ceros: 44969

cliente_activo:
  Min: 0.000
  Max: 1.000
  Media: 0.115
  Ceros: 44267

cobertura_beneficios:
  Min: 0.000
  Max: 3.000
  Media: 1.384
  Ceros: 12441


In [90]:
# =============================================================================
# VALIDACIÓN DE REGLAS DE NEGOCIO
# =============================================================================
def validar_reglas_negocio(df):
   print("\n=== VALIDACIÓN REGLAS DE NEGOCIO ===")
   
# Validación de reglas de negocio
def validate_business_rules(df):
   """Valida reglas de negocio específicas del sector crediticio"""
   
   print("Validación de reglas de negocio:")
   
   # Regla 1: Utilización no puede ser negativa
   util_negativa = (df['utilizacion_cupo'] < 0).sum()
   status_util = "OK" if util_negativa == 0 else "REVISAR"
   print(f"  Utilización negativa: {util_negativa} casos ({status_util})")
   
   # Regla 2: Capacidad pago muy alta es sospechosa
   pago_alto = (df['capacidad_pago'] > 2).sum()
   status_pago = "OK" if pago_alto < 100 else "REVISAR"
   print(f"  Capacidad pago >200%: {pago_alto} casos ({status_pago})")
   
   # Regla 3: Beneficios en rango válido
   benef_invalido = (df['cobertura_beneficios'] > 3).sum()
   status_benef = "OK" if benef_invalido == 0 else "REVISAR"
   print(f"  Beneficios inválidos: {benef_invalido} casos ({status_benef})")
   
   return {
       'utilizacion_negativa': util_negativa,
       'capacidad_pago_alta': pago_alto,
       'beneficios_invalidos': benef_invalido
   }
validar_reglas_negocio(train)

# %%


=== VALIDACIÓN REGLAS DE NEGOCIO ===


In [91]:

# =============================================================================
#   GUARDAR DATASETS FINALES
# =============================================================================
train.to_csv('data/processed/train_with_features.csv', index=False)
test.to_csv('data/processed/test_with_features.csv', index=False)

print(f"\n=== RESUMEN FINAL ===")
print(f"Total features creadas: {len(todas_features)}")
print(f"Variables utilización/comportamiento: 6")
print(f"Índices stress/actividad: 6") 
print(f"Variables beneficios Colsubsidio: 7")
print(f"Validación calidad completada")
print(f"Train final: {train.shape}")
print(f"Test final: {test.shape}")
print("Datos listos para modelado")


=== RESUMEN FINAL ===
Total features creadas: 19
Variables utilización/comportamiento: 6
Índices stress/actividad: 6
Variables beneficios Colsubsidio: 7
Validación calidad completada
Train final: (50001, 52)
Test final: (5001, 51)
Datos listos para modelado


In [92]:
# =============================================================================
#   CORRECCIONES FINALES
# =============================================================================
train = train.drop(['ratio_avances'], axis=1)
test = test.drop(['ratio_avances'], axis=1)

# Corregir capacidad_pago extrema
train['capacidad_pago'] = np.where(train['capacidad_pago'] > 5, 5, train['capacidad_pago'])
test['capacidad_pago'] = np.where(test['capacidad_pago'] > 5, 5, test['capacidad_pago'])

print("✅ Correcciones aplicadas")


✅ Correcciones aplicadas


In [93]:
# =============================================================================
#   GUARDAR DATASETS FINALES    
# =============================================================================
train.to_csv('data/processed/train_with_features.csv', index=False)
test.to_csv('data/processed/test_with_features.csv', index=False)

print(f"\nDatos con features guardados")
print(f"Train: {train.shape}")
print(f"Test: {test.shape}")
print("Listos para modelado")


Datos con features guardados
Train: (50001, 51)
Test: (5001, 50)
Listos para modelado
